In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *

yu.setpath('check_sensitivity_c3pt_to_c2pt')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

In [2]:
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    # factor_equal=np.mean(factor_equal)[None]
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [3]:
def makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunits=1,yunits=1,tcmin_rainbow=1,selection=None):
    if type(xunits)!=list:
        xunits=[xunits]*len(list_tf2ratio_fits)
    if type(yunits)!=list:
        yunits=[yunits]*len(list_tf2ratio_fits)
    
    fig, axs = yu.getFigAxs(len(list_tf2ratio_fits),4,Lrow=4,Lcol=6,sharex='col',sharey='row', gridspec_kw={'width_ratios': [3, 2, 2, 2]})
    irow=-1
    axs[irow,0].set_xlabel(r'$t_{\rm ins}-t_{s}/2$ [fm]')        
    axs[irow,1].set_xlabel(r'$t_{s}$ [fm]')
    axs[irow,2].set_xlabel(r'$t_{s}^{\rm low}$ [fm]')
    axs[irow,3].set_xlabel(r'Fit Prob.')
    
    ax=axs[irow,3]
    xticks=[1,3,10,30,90]
    ax.set_xlim([xticks[0]/2,150])
    ax.set_xscale('log')
    ax.set_xticks(xticks)
    ax.set_xticklabels([f'{x}%' for x in xticks])
    
    def get_fittype(tf2ratio_fits):
        tf2ratio,fits=tf2ratio_fits[:2]
        fitlabel,pars_jk,chi2_jk,Ndof=fits[0]
        if pars_jk.shape[1]==1:
            fittype='1st'
        elif len(list_tf2ratio_fits[irow])==3:
            fittype='2st2step'
        elif pars_jk.shape[1]==2:
            fittype='sum'
        else:
            fittype='2st'
        return fittype
    
    tfmin_ratio=1e10; tfmax_ratio=0
    tfmin_fit=1e10; tfmax_fit=0
    for irow in range(len(list_tf2ratio_fits)):
        xunit=xunits[irow]; yunit=yunits[irow]
        tf2ratio,fits=list_tf2ratio_fits[irow][:2]
        fittype=get_fittype(list_tf2ratio_fits[irow])
            
        pars_jk,probs_jk=yu.jackMA(fits)
        mean,err=yu.jackme(pars_jk)
        plt_y_quote,plt_yerr_quote=mean[0]*yunit,err[0]*yunit

        tfs=list(tf2ratio.keys()); tfs.sort()
        tfs_rainbow=[tf for tf in tfs if tf%2==0 and tf>=2*tcmin_rainbow and yu.jackme(tf2ratio[tf][:,tf//2])[1]*yunit<plt_yerr_quote*5]
        if len(tfs_rainbow)==0:
            tfs_rainbow=[tf for tf in tfs if tf%2==0 and tf>=2*tcmin_rainbow and yu.jackme(tf2ratio[tf][:,tf//2])[1]*yunit<plt_yerr_quote*20]
        # print(irow,tfs_rainbow)
        
        tcmins=list(set([fit[0][1] for fit in fits])); tcmins.sort()
        tfmins=list(set([fit[0][0] for fit in fits])); tfmins.sort()
        
        tfmin_ratio=min(tfmin_ratio,np.min(tfs_rainbow))
        tfmax_ratio=max(tfmax_ratio,np.max(tfs_rainbow))
        
        tfmin_fit=min(tfmin_fit,np.min(tfmins))
        tfmax_fit=max(tfmax_fit,np.max(tfmins))
        
    axs[0,0].set_xlim(np.array([-tfmax_ratio/2+(tcmin_rainbow-1),tfmax_ratio/2-(tcmin_rainbow-1)])*xunit)
    axs[0,1].set_xlim(np.array([tfmin_ratio-1,tfmax_ratio+3])*xunit)
    axs[0,2].set_xlim(np.array([tfmin_fit-1,tfmax_fit+1])*xunit)
            
    result=[]
    for irow in range(len(list_tf2ratio_fits)):
        xunit=xunits[irow]; yunit=yunits[irow]
        tf2ratio,fits=list_tf2ratio_fits[irow][:2]
        fittype=get_fittype(list_tf2ratio_fits[irow])
        
        pars_jk,probs_jk=yu.jackMA(fits)
        result.append(pars_jk)
        mean,err=yu.jackme(pars_jk)
        plt_y_quote,plt_yerr_quote=mean[0]*yunit,err[0]*yunit
        for icol in [0,1,2,3]:
            ax=axs[irow,icol]
            plt_x=ax.get_xlim(); plt_y=plt_y_quote; plt_yerr=plt_yerr_quote
            ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color='red',alpha=0.2,label=yu.un2str(plt_y,plt_yerr))  
        if fittype in ['1st']:
            axs[irow,0].set_ylim([plt_y_quote-plt_yerr_quote*16,plt_y_quote+plt_yerr_quote*16])
            
        tfs=list(tf2ratio.keys()); tfs.sort()
        tfs_rainbow=[tf for tf in tfs if tf%2==0 and tf>=2*tcmin_rainbow and yu.jackme(tf2ratio[tf][:,tf//2])[1]*yunit<plt_yerr_quote*5]
            
        ax=axs[irow,0]
        for itf,tf in enumerate(tfs_rainbow):
            mean,err=yu.jackme(tf2ratio[tf])
            tcs=np.arange(tcmin_rainbow,tf-tcmin_rainbow+1)
            plt_x=(tcs-tf/2+0.05*(itf-len(tfs_rainbow)/2))*xunit; plt_y=mean[tcs]*yunit; plt_yerr=err[tcs]*yunit
            ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors16[itf],fmt=yu.fmts16[itf])
            
        ax=axs[irow,1]
        for itf,tf in enumerate(tfs_rainbow):
            if fittype in ['1st','sum']:
                y_jk=tf2ratio[tf][:,tcmin_rainbow:tf-tcmin_rainbow+1]
                if np.all(y_jk[:,0]==y_jk[:,-1]): # check if symmetrized
                    y_jk=y_jk[:,:((y_jk.shape[1]+1)//2)]
                pars_jk,_,_=yu.doFit_const(y_jk)
                mean,err=yu.jackme(pars_jk)
            elif fittype in ['2st','2st2step']:
                mean,err=yu.jackme(tf2ratio[tf][:,tf//2])
            plt_x=(tf)*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors16[itf],fmt=yu.fmts16[itf])
            
            if fittype in ['1st','sum'] and tf+1 in tf2ratio.keys():
                y_jk=tf2ratio[tf+1][:,tcmin_rainbow:tf-tcmin_rainbow+2]
                if np.all(y_jk[:,0]==y_jk[:,-1]): # check if symmetrized
                    y_jk=y_jk[:,:((y_jk.shape[1]+1)//2)]
                pars_jk,_,_=yu.doFit_const(y_jk)
                mean,err=yu.jackme(pars_jk)
                plt_x=(tf+1)*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
                ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors16[itf+len(tfs_rainbow)],fmt=yu.fmts16[itf+len(tfs_rainbow)])
                
            
        ax=axs[irow,2]
        tcmins=list(set([fit[0][1] for fit in fits])); tcmins.sort()
        tfmins=list(set([fit[0][0] for fit in fits])); tfmins.sort()
        probs=np.mean(probs_jk,axis=0); inds=np.argsort(probs)
        ind_mpf_global=inds[-1]; fitlabel_mpf=fits[ind_mpf_global][0]
        # ind_mpf2_global=inds[-2]; fitlabel_mpf2=fits[ind_mpf2_global][0]
        for i_tfmin,tfmin in enumerate(tfmins):
            tfits=[fit for fit in fits if fit[0][0]==tfmin]
            pars_jk,probs_jk=yu.jackMA(tfits)
            ind_mpf=np.argmax(np.mean(probs_jk,axis=0))
            
            (tfmin,tcmin),pars_jk,chi2_jk,Ndof=tfits[ind_mpf]
            mean,err=yu.jackme(pars_jk)
            
            plt_x=(tfmin)*xunit; plt_y=mean[0]*yunit; plt_yerr=err[0]*yunit
            ax.errorbar(plt_x,plt_y,plt_yerr,color='r',mfc='white' if (tfmin,tcmin)==fitlabel_mpf else None) 
        
        if fittype in ['2st','2st2step']:
            pars_jk,probs_jk=yu.jackMA(fits)
            ind_mpf=np.argmax(np.mean(probs_jk,axis=0))
            (tfmin,tcmin),pars_jk,chi2_jk,Ndof=fits[ind_mpf]
            ax=axs[irow,0]
            t_cut=tcmin
            for itf,tf in enumerate(tfs_rainbow):
                if tf<tfmin:
                    continue
                tcs=np.arange(t_cut,tf-t_cut,0.1)
                if fittype in ['2st2step']:
                    pars_jk_meff2st=list_tf2ratio_fits[irow][2]
                    t=np.array([yu.func_ratio_2st(tf,tcs,pars[0],pars_2pt[1],pars_2pt[2],pars[1],pars[2]) for pars, pars_2pt in zip(pars_jk,pars_jk_meff2st)])
                else:
                    t=np.array([yu.func_ratio_2st(tf,tcs,*pars) for pars in pars_jk])
                mean,err=yu.jackme(t)
                plt_x=(tcs-tf//2)*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
                ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color=yu.colors16[itf],alpha=0.2)   
            ax=axs[irow,1]
            tfs=np.arange(ax.get_xlim()[0]/xunit,ax.get_xlim()[1]/xunit,0.1)
            if fittype in ['2st2step']:
                pars_jk_meff2st=list_tf2ratio_fits[irow][2]
                t=np.array([yu.func_ratio_2st(tfs,tfs/2,pars[0],pars_2pt[1],pars_2pt[2],pars[1],pars[2]) for pars, pars_2pt in zip(pars_jk,pars_jk_meff2st)])
            else:
                t=np.array([yu.func_ratio_2st(tfs,tfs/2,*pars) for pars in pars_jk])
            mean,err=yu.jackme(t)
            plt_x=tfs*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color='grey',alpha=0.2)   
            
        ax=axs[irow,3]
        pars_jk,probs_jk=yu.jackMA(fits)
        for ifit, fit in enumerate(fits):
            (tfmin,tcmin),pars_jk,chi2_jk,Ndof=fit
            prob=np.mean(probs_jk,axis=0)[ifit]
            if prob<1/100:
                continue
            mean,err=yu.jackme(pars_jk)
            plt_x=(prob)*100; plt_y=mean[0]*yunit; plt_yerr=err[0]*yunit
            ax.errorbar(plt_x,plt_y,plt_yerr,color='r',mfc='white' if (tfmin,tcmin)==fitlabel_mpf else None,
                        label=f'{int(probs[ind_mpf_global]*100)}%; {(tfmin,tcmin)}' if (tfmin,tcmin)==fitlabel_mpf else None)
        ax.legend(fontsize=16)
        
    return fig,axs,result

In [4]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,0),lambda x:modify(x,1),lambda x:modify(x,2),lambda x:modify(modify(modify(x,0),1),2)]
columns=['normal','0err_m','0err_dE','0err_rc1','0err']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunits=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

,normal,0err_m,0err_dE,0err_rc1,0err
j+;conn_B64,0.386(15),0.386(15),0.3860(91),0.386(15),0.3860(91)
j+;conn_C80,0.329(17),0.329(17),0.3285(53),0.329(17),0.3285(53)
j+;conn_D96,0.327(16),0.327(16),0.3271(39),0.327(16),0.3271(39)
j-;conn_B64,0.1597(90),0.1597(90),0.1597(80),0.1597(90),0.1597(80)
j-;conn_C80,0.1405(73),0.1405(73),0.1405(59),0.1405(73),0.1405(59)
j-;conn_D96,0.1338(74),0.1338(74),0.1338(28),0.1338(74),0.1338(28)


In [5]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    # t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    t[:,i]=t[:,i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,1,shift=-1),lambda x:modify(x,1,shift=0),lambda x:modify(x,1,shift=+1)]
columns=['normal','dE-1','dE','dE+1']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunits=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

,normal,dE-1,dE,dE+1
j+;conn_B64,0.386(15),0.373(17),0.386(15),0.397(13)
j+;conn_C80,0.329(17),0.309(21),0.329(17),0.344(14)
j+;conn_D96,0.327(16),0.310(19),0.327(16),0.341(13)
j-;conn_B64,0.1597(90),0.154(10),0.1597(90),0.1642(82)
j-;conn_C80,0.1405(73),0.1349(85),0.1405(73),0.1451(65)
j-;conn_D96,0.1338(74),0.1262(88),0.1338(74),0.1401(63)


In [6]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    # t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    t[:,i]=t[:,i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,0,shift=-10),lambda x:modify(x,0,shift=0),lambda x:modify(x,0,shift=+10)]
columns=['normal','m-10','m','m+10']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunits=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

# the parameterization of the ratio is actually independent of m

,normal,m-10,m,m+10
j+;conn_B64,0.386(15),0.386(15),0.386(15),0.386(15)
j+;conn_C80,0.329(17),0.329(17),0.329(17),0.329(17)
j+;conn_D96,0.327(16),0.327(16),0.327(16),0.327(16)
j-;conn_B64,0.1597(90),0.1597(90),0.1597(90),0.1597(90)
j-;conn_C80,0.1405(73),0.1405(73),0.1405(73),0.1405(73)
j-;conn_D96,0.1338(74),0.1338(74),0.1338(74),0.1338(74)
